In [1]:
from datetime import datetime
import time
import gspread
import pandas as pd
import numpy as np
from oauth2client.service_account import ServiceAccountCredentials
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)
client = gspread.authorize(creds)

In [2]:
def days_sum(day, change):
    date = day + change
    return date

In [3]:
sh = client.open_by_key('1nJmpNVlyePB09EnMFET_CAsLccu6hs-IP3fsRBG0UXQ')
worksheet = sh.get_worksheet(2)
data = worksheet.get_all_values()
draws_df = pd.DataFrame(data)
draws_df = draws_df[[0, 1, 2, 3, 4, 5, 6, 7]]
draws_df.drop([0, 1, 2, 3], inplace = True)
draws_df.columns = ['Sorteos', 'Nro1', 'Nro2', 'Nro3', 'Nro4', 'Nro5', 'Star_1', 'Star_2']
draws_df = draws_df.replace(r'^s*$', float('NaN'), regex = True)
draws_df.dropna(how = 'all', inplace = True)
draws_df.reset_index(drop = True, inplace = True)

In [4]:
entry_point = time.mktime((2004, 2, 6, 0, 0, 0, 0, 0, 0)) # reference for math - first Friday of February 2004
Tuesday = time.mktime((2004, 2, 10, 0, 0, 0, 0, 0, 0)) # reference for math with four days of difference
first_draw = time.mktime((2004, 2, 13, 0, 0, 0, 0, 0, 0)) # this is the first draws of Euro Millions
change_draw = time.mktime((2011, 5, 6, 0, 0, 0, 0, 0, 0)) # this is the Last draw of the old rule: one draw per week
next_week = first_draw - entry_point
next_Tuesday = Tuesday - entry_point
Tuesday_Friday = first_draw - Tuesday

In [5]:
dates = []
dates1 = []

for i in range(378):
    entry_point = days_sum(entry_point, next_week)
    dates.append(entry_point)

for i in range(379, len(draws_df) + 1):
    if i % 2:
        change_draw = days_sum(change_draw, next_Tuesday)
    else:
        change_draw = days_sum(change_draw, Tuesday_Friday)
    dates1.append(change_draw)

df = pd.DataFrame(list(map(datetime.fromtimestamp, dates)))
df1 = pd.DataFrame(list(map(datetime.fromtimestamp, dates1)))
frames = [df, df1]
result = pd.concat(frames, ignore_index=True)
result = result.rename(columns={0:'Dates'})

In [6]:
df_clean = {
    'Sorteos': "string",
    'Nro1': "int64",
    'Nro2': "int64",
    'Nro3': "int64",
    'Nro4': "int64",
    'Nro5': "int64",
    'Star_1': "int64",
    'Star_2': "int64"
}
draws_df = draws_df.astype(df_clean)
draws_df = pd.concat([result, draws_df], axis=1, join='inner')
draws_df

,Dates,Sorteos,Nro1,Nro2,Nro3,Nro4,Nro5,Star_1,Star_2
0,2004-02-13,1,16,29,32,36,41,7,9
1,2004-02-20,2,7,13,39,47,50,2,5
2,2004-02-27,3,14,18,19,31,37,4,5
3,2004-03-05,4,4,7,33,37,39,1,5
4,2004-03-12,5,15,24,28,44,47,4,5
...,...,...,...,...,...,...,...,...,...
1598,2023-01-17,1599,18,21,30,46,50,2,6
1599,2023-01-20,1600,3,6,13,40,42,7,11
1600,2023-01-24,1601,11,27,41,43,47,7,11
1601,2023-01-27,1602,8,9,15,19,31,10,12


In [7]:
draws_df.dtypes

Dates      datetime64[ns]
Sorteos            string
Nro1                int64
Nro2                int64
Nro3                int64
Nro4                int64
Nro5                int64
Star_1              int64
Star_2              int64
dtype: object

In [8]:
draws_df.describe()

,Nro1,Nro2,Nro3,Nro4,Nro5,Star_1,Star_2
count,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000
mean,8.406114,16.837804,25.251404,34.009981,42.519027,3.940112,8.008110
std,6.611883,8.189616,8.674071,8.425625,6.750973,2.490463,2.548272
min,1.000000,2.000000,5.000000,9.000000,10.000000,1.000000,2.000000
25%,3.000000,10.000000,19.000000,28.000000,39.000000,2.000000,6.000000
50%,7.000000,16.000000,25.000000,35.000000,44.000000,3.000000,8.000000
75%,12.000000,23.000000,32.000000,40.000000,48.000000,6.000000,10.000000
max,38.000000,44.000000,47.000000,49.000000,50.000000,11.000000,12.000000


In [9]:
draws_df.to_csv('base_de_datos.csv', index=False)

In [10]:
# row_189 = draws_df[draws_df["Sorteos"] == "378"]
# print(row_189)